In [1]:
import keras.backend as K
from keras import Input, Model, Sequential
from keras.layers import *
from keras.optimizers import Adam, RMSprop
from keras.callbacks import *

/home/ing-luca/.local/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def split_data(dataset, timesteps):
    D = dataset.shape[1]
    if D < timesteps:
        return None
    elif D == timesteps:
        return dataset
    else:
        splitted_data, remaining_data = np.hsplit(dataset, [timesteps])
        remaining_data = split_data(remaining_data, timesteps)
        if remaining_data is not None:
            return np.vstack([splitted_data, remaining_data])
        return splitted_data

In [85]:
normalized_transactions_filepath = "../datasets/berka_dataset/usable/normalized_transactions.npy"
generated_transactions_filepath = "../datasets/generated_data.npy"

dataset_1_filepath = "wgan/2018-04-04_10-50-16/generated_datasets/last.npy"

timesteps = 100
elements = 50000

real_data = np.load(normalized_transactions_filepath)
real_data = split_data(real_data, timesteps)
real_data = real_data[np.std(real_data, 1) > float(1e-7)][:elements]

fake_data = np.load(generated_transactions_filepath)[:elements]

real_data.shape, fake_data.shape

((50000, 100), (50000, 100))

In [87]:
split = 0.2

x_train = np.vstack([real_data, fake_data]) 
y_train = np.vstack([np.zeros((elements, 1)), np.ones((elements, 1))])

shuffled_indexes = np.random.permutation(x_train.shape[0])

x_train = x_train[shuffled_indexes]
y_train = y_train[shuffled_indexes]

split_index = int(2*elements*split)

x_test, x_train = x_train[:split_index], x_train[split_index:]
y_test, y_train = y_train[:split_index], y_train[split_index:]


x_train.shape, y_train.shape, x_test.shape, y_test.shape

((80000, 100), (80000, 1), (20000, 100), (20000, 1))

In [88]:
model_inputs = Input((timesteps,))
classified = model_inputs

classified = Lambda(lambda x: K.expand_dims(x))(classified)
while classified.shape[1] > 1:
    classified = Conv1D(32, 3, activation='relu', padding='same')(classified)
    classified = MaxPooling1D(2, padding='same')(classified)
classified = Flatten()(classified)
classified = Dense(15, activation='relu')(classified)
classified = Dense(5, activation='relu')(classified)
classified = Dense(1, activation = 'sigmoid')(classified) 

classifier = Model(model_inputs, classified, 'classifier')
classifier.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [89]:
classifier.fit(x=x_train, y=y_train, validation_data=(x_test, y_test), epochs=5)

Train on 80000 samples, validate on 20000 samples
Epoch 1/5
80000/80000 [==============================] - 8s 101us/step - loss: 0.4327 - acc: 0.9211 - val_loss: 0.2886 - val_acc: 0.9585
Epoch 2/5
80000/80000 [==============================] - 7s 86us/step - loss: 0.2310 - acc: 0.9617 - val_loss: 0.1916 - val_acc: 0.9651
Epoch 3/5
80000/80000 [==============================] - 7s 88us/step - loss: 0.1699 - acc: 0.9669 - val_loss: 0.1543 - val_acc: 0.9686
Epoch 4/5
80000/80000 [==============================] - 7s 88us/step - loss: 0.1439 - acc: 0.9698 - val_loss: 0.1373 - val_acc: 0.9690
Epoch 5/5
80000/80000 [==============================] - 7s 88us/step - loss: 0.1308 - acc: 0.9717 - val_loss: 0.1275 - val_acc: 0.9716
